In [1]:
import numpy as np
import scipy.sparse as sparse
import random
import hashlib

# Load in Data

In [2]:
data = np.load('Data/user_movie_rating.npy')[:,0:2]

In [3]:
col = data[:, 0]
row = data[:, 1]

n_col = col.max() + 1
n_row = row.max() + 1 

M = sparse.coo_matrix((np.ones(len(data)), (row, col)), shape=(n_row, n_col)).tocsr()
small_M  = M[:,0:1000]

In [4]:
perm = 100 
perm_seeds = []
row_len,_ = small_M.get_shape()
seed = 1000
for i in range(perm):
    perm_seeds.append(int(hashlib.sha256(str(seed+i).encode()).hexdigest(),16)%4294967295)


In [5]:
from tqdm import tqdm


def minhashing(M):
    signature = []
    index_perm = np.arange(M.shape[0])#reshuffeling the same index array
    sig_len = M.shape[1]-1
    for seed in tqdm(perm_seeds):
        random.seed(seed)
        np.random.shuffle(index_perm)
        perm_M = M[index_perm[:1000],1:] #Just take the top 1000 because it is very likely for each user to be in the first thousand movies at least once
        non_zero = perm_M.nonzero()
        sig = non_zero[0][np.unique(non_zero[1], return_index=True)[1]]
        if len(sig)==sig_len:
            signature.append(sig)        
    return signature
        

        


In [6]:
a = minhashing(M)

100%|██████████| 100/100 [00:21<00:00,  4.64it/s]


In [1]:
u = np.array(a).T.tolist()

NameError: name 'np' is not defined

In [ ]:
len(u)

In [ ]:
len(u[0])

In [ ]:
def LSH(M,b):
    #Create Bands and Hashes
    n_users = len(M)
    sign_len = len(M[0])
    n_rows = int(sign_len/b)
    buckets = [ [] for _ in tqdm(range(int(n_users*b))) ]
    candidates = []
    for i in range(n_users):
        for band in range(b):
            hash_num = (int((sha256(f"{u[i][n_rows*band:(n_rows*(band + 1) - 1)]}".encode())).hexdigest(), 16)%int(n_users*b))
            buckets[hash_num].append(i)
            if len(buckets[hash_num])==2:
                candidates.append(hash_num)
            
    return buckets, candidates     
# implement hash table to check for identity in linear time

        


In [ ]:
resultado, candidates = LSH(u,5)

In [ ]:
len(candidates)

In [ ]:
len(resultado[candidates[0]])

In [ ]:
MT = M.T

In [ ]:
def Jaccard_similarity(u1,u2,M):
    movies_u1 = set(MT[u1].nonzero()[1])
    movies_u2 = set(MT[u2].nonzero()[1])
    similarity = len(movies_u1.intersection(movies_u2))/len(set(movies_u1).union(movies_u2))
    return similarity

In [ ]:
def get_pair_similarity(candidates, buckets,M):
    pair_similarity = {}
    for i in tqdm(range(len(candidates))):
        for j in range(len(buckets[candidates[i]]) - 1):
            for z in range(j + 1, len(buckets[candidates[i]])):
                u1 = buckets[candidates[i]][j]
                u2 = buckets[candidates[i]][z]
                if f"{u1},{u2}" not in pair_similarity and f"{u2},{u1}" not in pair_similarity:
                    similarity = Jaccard_similarity(u1,u2,M)
                    pair_similarity[f"{u1},{u2}"] = similarity
    return pair_similarity     

In [ ]:
similiarities = get_pair_similarity(candidates, resultado, MT)